In [1]:
import numpy as np

In [2]:
np.random.seed(42)

In [7]:
n_features = 10
max_features = 12

feature_idxs = np.random.choice(n_features, max_features, replace=False)
feature_idxs

ValueError: Cannot take a larger sample than population when 'replace=False'